In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
#df = pd.read_csv("audible/Features_5too10hz_aud.csv")
#df = pd.read_csv("audible/Features_10too15hz_aud.csv")
#df = pd.read_csv("audible/Features_15too20hz_aud.csv")
df = pd.read_csv("audible/Features10too200hz_aud.csv")



#frames =[df3,df4,df5,df6,df7]
#df = pd.concat(frames)


#dfs = df[(df.modeID != 'audible')]
df


C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


channel_1_mav  channel_1_mav2  channel_1_rms  channel_1_iav  \
0          0.034982        0.033315       0.047836     117.153433   
1          0.028157        0.026247       0.040353      63.465079   
2          0.035103        0.034455       0.049304     142.411421   
3          0.032311        0.032380       0.044599     134.576651   
4          0.030766        0.030669       0.044142     144.633076   
...             ...             ...            ...            ...   
7374       0.035106        0.032929       0.049159      65.542070   
7375       0.033092        0.031553       0.046681      64.628148   
7376       0.033499        0.030709       0.050080      56.780836   
7377       0.036185        0.035573       0.052843      70.705897   
7378       0.035235        0.033532       0.050398      71.103609   

      channel_1_ssi  channel_1_var  channel_1_wl  channel_1_iemg  \
0          7.663511       0.002288    131.601251      117.153433   
1          3.670272       0.001628     71.151924       63.465079   
2          9.862262       0.002431    160.962810      142.411421   
3          8.284614       0.001989    151.073407      134.576651   
4          9.159818       0.001948    161.532869      144.633076   
...             ...            ...           ...             ...   
7374       4.511806       0.002417     70.545085       65.542070   
7375       4.255896       0.002179     69.438232       64.628148   
7376       4.251053       0.002508     61.153328       56.780836   
7377       5.456400       0.002792     75.374112       70.705897   
7378       5.125579       0.002540     77.832307       71.103609   

      channel_1_aac  channel_1_zc  ...  channel_6_aac  channel_6_zc  \
0          0.039296          1239  ...       0.043452          1150   
1          0.031567           861  ...       0.032478           749   
2          0.039675          1552  ...       0.050138          1382   
3          0.036272          1551  ...       0.054896          1449   
4          0.034361          1743  ...       0.050953          1611   
...             ...           ...  ...            ...           ...   
7374       0.037785           660  ...       0.007766           697   
7375       0.035555           698  ...       0.007459           737   
7376       0.036079           622  ...       0.007684           651   
7377       0.038574           662  ...       0.008002           718   
7378       0.038569           749  ...       0.007368           743   

      channel_6_ssc  channel_6_wamp  channel_6_medf  channel_6_meanf  userID  \
0              1395               2       94.594595       111.041462       1   
1               930               1      100.446429       111.273560       1   
2              1683               7       95.049505       109.108856       1   
3              1761               9       91.807229       109.532973       1   
4              1922               6       97.867804       112.357899       1   
...             ...             ...             ...              ...     ...   
7374            851               0      105.405405       113.118444       8   
7375            918               0       89.690722       108.367480       8   
7376            812               0       75.000000       102.275545       8   
7377            889               0       88.144330       105.494360       8   
7378            933               0       82.500000       106.567297       8   

      sessionID     uttID   modeID  
0          1001  10010001  audible  
1          1001  10010002  audible  
2          1001  10010003  audible  
3          1001  10010004  audible  
4          1001  10010005  audible  
...         ...       ...      ...  
7374       8101  81010506  audible  
7375       8101  81010507  audible  
7376       8101  81010508  audible  
7377       8101  81010509  audible  
7378       8101  81010510  audible  

[7379 rows x 88 columns]

In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['userID']
    y_test = X_test['userID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['userID']
    y_test = X_test['userID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [3]:
def lda():
    ids = [2001,2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,2011, 2012, 2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,
        2023,2024,2025,2026,2027,2028, 2029, 2030,2031, 2032,7002,8001,8002,8003,8004,8005,8006,8007,8008,8009, 8010, 8011,8012,8013,8014,8015, 8016, 8017, 8018,8019, 8101]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = KNeighborsClassifier(n_neighbors=6)
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))
    print(scores.std())

In [4]:
lda()

Accuracy for session [2001] is 1.0
Accuracy for session [2002] is 1.0
Accuracy for session [2003] is 0.9933333333333333
Accuracy for session [2004] is 1.0
Accuracy for session [2005] is 0.98
Accuracy for session [2006] is 1.0
Accuracy for session [2007] is 0.9933333333333333
Accuracy for session [2008] is 0.9933333333333333
Accuracy for session [2009] is 1.0
Accuracy for session [2010] is 1.0
Accuracy for session [2011] is 1.0
Accuracy for session [2012] is 0.9933333333333333
Accuracy for session [2013] is 1.0
Accuracy for session [2014] is 0.98
Accuracy for session [2015] is 1.0
Accuracy for session [2016] is 1.0
Accuracy for session [2017] is 1.0
Accuracy for session [2018] is 1.0
Accuracy for session [2019] is 1.0
Accuracy for session [2020] is 0.98
Accuracy for session [2021] is 1.0
Accuracy for session [2022] is 1.0
Accuracy for session [2023] is 1.0
Accuracy for session [2024] is 1.0
Accuracy for session [2025] is 1.0
Accuracy for session [2026] is 0.33
Accuracy for session [2027